## INTRO



In [50]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## READ_CSV

In [51]:
df = pd.read_csv('atuacoes_2.csv', encoding='UTF-8', nrows=200) #, dtype=str, nrows=1000)

df['ANO-FIM'] = df['ANO-FIM'].fillna(2024)
df['FLAG-DEDICACAO-EXCLUSIVA'] = df['FLAG-DEDICACAO-EXCLUSIVA'].replace('NAO', False).replace('SIM', True)

df = df.astype({
    'NRO-ID-CNPQ': int,
    'CODIGO-INSTITUICAO': str,
    'NOME-INSTITUICAO': str,
    'ANO-INICIO': int,
    'ANO-FIM': int,
    'ENQUADRAMENTO-FUNCIONAL': str,
    'TIPO-DE-VINCULO': str,
    'OUTRAS-INFORMACOES': str,
    'CARGA-HORARIA-SEMANAL': float,
    'FLAG-DEDICACAO-EXCLUSIVA': bool
})

df = df.fillna(' ')
df2 = df.copy(deep=True)

#df

In [52]:
# LISTA DE IES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_filtered = df_uni[['NOME_DA_IES', 'SIGLA']]
df_uni_filtered = df_uni_filtered.fillna('')
df_uni_filtered = df_uni_filtered.sort_values(['NOME_DA_IES'], ascending=False)
df_uni_filtered

,NOME_DA_IES,SIGLA
862,iCEV - Instituto de Ensino Superior,
2205,VOAS - INSTITUTO DE APRENDIZAGEM SUPERIOR,VOAS
2873,Universidade do Estado de Mato Grosso Carlos A...,UNEMAT
...,...,...
30,ACADEMIA DE POLÍCIA MILITAR DE MINAS GERAIS,APM
3933,ACADEMIA DA FORÇA AÉREA,AFA
2106,ABEU - CENTRO UNIVERSITÁRIO,UNIABEU


## PRE-PROCESSAMENTO

In [53]:
def clean_names(df, column_name):

    # Remover '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remover todos os caracteres nao alfa-numericos, exceto whitespace
    df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)

    # Normalizar a string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remover stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Strip whitespace
    df[column_name] = df[column_name].apply(lambda x: x.strip())

    return df

In [54]:
# Lista de Curriculo Lattes

# Tratar os nomes da NOME-INSTITUICAO
df2 = df2.apply(lambda x: x.astype(str).str.upper())
df2 = clean_names(df2, 'NOME-INSTITUICAO')

# Dropar linhas com IES < 50
#df2 = df2[df2.groupby('NOME-INSTITUICAO')['NOME-INSTITUICAO'].transform('count').ge(50)]

# Split df: educacao & outros
df2_edu = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['LIVRE', 'COLABORADOR', 'PROFESSOR_VISITANTE'])].copy(deep=True)
df2_outros = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['SERVIDOR_PUBLICO', 'CELETISTA', 'NAN', 'OUTRO'])].copy(deep=True)

df2_edu

,NRO-ID-CNPQ,CODIGO-INSTITUICAO,NOME-INSTITUICAO,ANO-INICIO,ANO-FIM,ENQUADRAMENTO-FUNCIONAL,TIPO-DE-VINCULO,OUTRAS-INFORMACOES,CARGA-HORARIA-SEMANAL,FLAG-DEDICACAO-EXCLUSIVA
0,3397219421632200,000100000991,HOSPITAL UNIVERSITARIO PROFESSOR ALBERTO ANTUNES,2011,2015,LIVRE,LIVRE,ANALISTA JÚNIOR III,40.0,FALSE
1,3397219421632200,000100000991,HOSPITAL UNIVERSITARIO PROFESSOR ALBERTO ANTUNES,2016,2017,LIVRE,LIVRE,SECRETÁRIO EXECUTIVO,30.0,FALSE
2,3397219421632200,000200000993,SECRETARIA ESTADO EDUCACAO ALAGOAS,2014,2017,LIVRE,LIVRE,PROFESSOR MONITOR DA DISCIPLINA DE GEOGRAFIA,10.0,FALSE
...,...,...,...,...,...,...,...,...,...,...
194,3352296527889230,000600000990,LARVI,2017,2017,LIVRE,LIVRE,ASSISTENTE DE LARVICULTURA,,FALSE
197,3380808284395942,000500000999,ESCOLA UNIVERSO INFANTIL,2008,2009,LIVRE,LIVRE,PROFESSORA DE EDUCAÇÃO INFANTIL E SUPERVISORA,,TRUE
199,3380808284395942,HAPE00000003,PREFEITURA MUNICIPAL JACAREACANGA,2012,2013,LIVRE,LIVRE,COORDENADOR PEDAGOGICO DO CRAS,40.0,TRUE


In [55]:
# Lista de IES
df_uni_filtered = df_uni_filtered.apply(lambda x: x.astype(str).str.upper())
df_uni_filtered = clean_names(df_uni_filtered, 'NOME_DA_IES')

# merge colunas NOME e SIGLA
df_uni_filtered['NOME_E_SIGLA'] = df_uni_filtered['SIGLA'] + ' ' + df_uni_filtered['NOME_DA_IES']
df_uni_filtered

,NOME_DA_IES,SIGLA,NOME_E_SIGLA
862,ICEV INSTITUTO ENSINO SUPERIOR,,ICEV INSTITUTO ENSINO SUPERIOR
2205,VOAS INSTITUTO APRENDIZAGEM SUPERIOR,VOAS,VOAS VOAS INSTITUTO APRENDIZAGEM SUPERIOR
2873,UNIVERSIDADE ESTADO MATO GROSSO CARLOS ALBERTO...,UNEMAT,UNEMAT UNIVERSIDADE ESTADO MATO GROSSO CARLOS ...
...,...,...,...
30,ACADEMIA POLICIA MILITAR MINAS GERAIS,APM,APM ACADEMIA POLICIA MILITAR MINAS GERAIS
3933,ACADEMIA FORCA AEREA,AFA,AFA ACADEMIA FORCA AEREA
2106,ABEU CENTRO UNIVERSITARIO,UNIABEU,UNIABEU ABEU CENTRO UNIVERSITARIO


In [56]:
# Split lista de IES por inicio do nome
df_uni_filtered.sort_values(by=["NOME_DA_IES"], ascending=True)

df_uni_outros = df_uni_filtered.copy(deep=True)

df_uni_instituto = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')].index, inplace=True)

df_uni_faculdades = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')].index, inplace=True)

df_uni_escola = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')].index, inplace=True)

df_uni_universidade = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')].index, inplace=True)

df_uni_instituicao = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')].index, inplace=True)

# List
# df_uni_outros, df_uni_instituto, df_uni_faculdades, df_uni_escola, df_uni_universidade, df_uni_instituicao

## NLP


### Cosine

In [57]:
def get_uni_list_df_for_tfidf(nome_instituicao):

    if (nome_instituicao in ('INSTITUTO')):
        return tfidf_matrix_instituto
    elif (nome_instituicao in ('FACULDADE')):
        return tfidf_matrix_faculdades
    elif (nome_instituicao in ('ESCOLA')):
        return tfidf_matrix_escola
    elif (nome_instituicao in ('UNIVERSIDADE')):
        return tfidf_matrix_universidade
    elif (nome_instituicao in ('INSTITUICAO')):
        return tfidf_matrix_instituicao

    return tfidf_matrix_outros


def get_uni_list_df_for_sigla(nome_instituicao):

    if (nome_instituicao in ('INSTITUTO')):
        return df_uni_instituto
    elif (nome_instituicao in ('FACULDADE')):
        return df_uni_faculdades
    elif (nome_instituicao in ('ESCOLA')):
        return df_uni_escola
    elif (nome_instituicao in ('UNIVERSIDADE')):
        return df_uni_universidade
    elif (nome_instituicao in ('INSTITUICAO')):
        return df_uni_instituicao

    return df_uni_outros


def iterate_sigla(nome_instituicao, df_for_search):

    matched_rows = []

    print(nome_instituicao)

    for index, row in df_for_search.iterrows():

        if row['SIGLA']:

            pattern = r'\b' + row['SIGLA'] + r'\b'
            if re.search(pattern, nome_instituicao):
                matched_rows.append(row['SIGLA'])
                print("Matched SIGLA: ", row['SIGLA'], row['NOME_DA_IES'])

            # dos diversos matches, qual mais faz sentido?
            #if len(matched_rows) > 1:

    return ",".join(str(element) for element in matched_rows)

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

threshold = 0.8

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_instituto = vectorizer.transform(df_uni_instituto['NOME_DA_IES'])
tfidf_matrix_faculdades = vectorizer.transform(df_uni_faculdades['NOME_DA_IES'])
tfidf_matrix_escola = vectorizer.transform(df_uni_escola['NOME_DA_IES'])
tfidf_matrix_universidade = vectorizer.transform(df_uni_universidade['NOME_DA_IES'])
tfidf_matrix_instituicao = vectorizer.transform(df_uni_instituicao['NOME_DA_IES'])
tfidf_matrix_outros = vectorizer.transform(df_uni_outros['NOME_DA_IES'])

# Nova Coluna com Resultado
df2_edu['nome_corrigido'] = ''
df2_edu['sigla_detectado'] = ''


# PLN
for index, row in df2_edu.iterrows():

    nome_instituicao = row['NOME-INSTITUICAO']
    print(nome_instituicao)

    which_df_to_search = get_uni_list_df_for_sigla(nome_instituicao)

    # Ver se contém a sigla
    sigla_detectado = iterate_sigla(nome_instituicao, which_df_to_search)
    if sigla_detectado:
        df2_edu['sigla_detectado'] = sigla_detectado

    # Se nao contem a sigla, procurar por nome completo
    else:
        vector_comparacao = vectorizer.transform([nome_instituicao])

        which_tfidf_to_search = get_uni_list_df_for_tfidf(nome_instituicao)
        similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_universidades)

        max_value = np.max(similaridade)
        max_index = np.argmax(similaridade)

        nome_corrigido = df_uni_filtered.iloc[max_index]['NOME_DA_IES']

        if (max_value >= threshold):
            df2_edu.at[index, 'nome_corrigido'] = nome_corrigido
        else:
            df2_edu.at[index, 'nome_corrigido'] = ''

    print("==================")

HOSPITAL UNIVERSITARIO PROFESSOR ALBERTO ANTUNES
HOSPITAL UNIVERSITARIO PROFESSOR ALBERTO ANTUNES
SECRETARIA ESTADO EDUCACAO ALAGOAS
UNIVERSIDADE ESTADO PARA
EDITORA ABRIL
UNYLEYA EDITORA CURSOS S A
CENTRO UNIVERSITARIO FAMETRO
FAMETRO CENTRO UNIVERSITARIO CEUNI FAMETRO
UNIAO ESCOLAS SUPERIORES RONDONOPOLIS
GOVERNO ESTADO MINAS GERAIS
ENDOIMAGEM SERVICOS MEDICOS LTDA
AMIU BOTAFOGO
UNIVERSIDADE PAULISTA
MARTINS COMERCIO SERVICOS DISTRIBUICAO S A
SUPERMERCADO BRASIL NORTE LTDA
COMERCIAL REAL CARNES LTDA
LANCHONETE SORVETERIA PISTACHIOS
MEISTER S A
DEPARTAMENTO MUNICIPAL AGUA ESGOTO UBERLANDIA
SORVETES LIDER
CONTA FACIL PRESTACAO SERVICOS LTDA
DROGA RAIA S A
MEIZLER BIOPHARMA LTDA
DARROW LABORATORIOS
MOKSHA8 REPRESENTACOES
SANOFI AVENTIS FARMACEUTICA
LABORATORIO QUIMICO FARMACEUTICO BERGAMO
FACULDADES INTEGRADAS TORRICELLI
CONSELHO REGIONAL FARMACIA ESTADO SAO PAULO
UNIVERSIDADE FEDERAL PIAUI
UNIVERSIDADE FEDERAL PIAUI
UNIVERSIDADE FEDERAL PIAUI CAMPUS MINISTRO REIS VELLOSO
UNIVERSIDADE F

In [ ]:
# oq foi alterado
pd.set_option('display.max_rows', 10)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido']])
pd.set_option('display.max_rows', 6)